In [5]:
# z = x^4 - 5*(y^2)*(x*2)

# 1) v1 = x*x
# 2) v2 = v1 * v1
# 3) v3 = -5 * y * y
# 4) z - v2 = v3 * v1

# witness : [1,z,x,y,v1,v2,v3]

import numpy as np

L = np.array([
    [0,0,1,0,0,0,0],
    [0,0,0,0,1,0,0],
    [0,0,0,-5,0,0,0],
    [0,0,0,0,0,0,1]
])

R = np.array([
    [0,0,1,0,0,0,0],
    [0,0,0,0,1,0,0],
    [0,0,0,1,0,0,0],
    [0,0,0,0,1,0,0]
])

O = np.array([
    [0,0,0,0,1,0,0],
    [0,0,0,0,0,1,0],
    [0,0,0,0,0,0,1],
    [0,1,0,0,0,-1,0]
])

x = 4
y = -2
v1 = x * x
v2 = v1 * v1        # x^4
v3 = -5*y * y
z = v3*v1 + v2    # -5y^2 * x^2

# witness
a = np.array([1, z, x, y, v1, v2, v3])

assert all(np.equal(np.matmul(L, a) * np.matmul(R, a), np.matmul(O, a))), "not equal"

In [6]:
P =79

# galois field does not take negative values
L = (L + P) % P
R = (R +P) % P
O = (O+P) % P

print(L)
print(R)
print(O)

[[ 0  0  1  0  0  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  0  0 74  0  0  0]
 [ 0  0  0  0  0  0  1]]
[[0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]]
[[ 0  0  0  0  1  0  0]
 [ 0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  1]
 [ 0  1  0  0  0 78  0]]


In [12]:
import galois

GF = galois.GF(P)

L_galois = GF(L)
R_galois = GF(R)
O_galois = GF(O)


print(L_galois)
print(R_galois)
print(O_galois)

x = GF(4)
y = GF(-2 + P)
v1=  x*x
v2 = v1* v1
v3 = GF(-5+ P) * y * y
z = v3 * v1 + v2

witness = GF(np.array([1,z,x,y,v1,v2,v3]))

assert all(np.equal(np.matmul(L_galois, witness) * np.matmul(R_galois,witness), np.matmul(O_galois , witness))), "not equal"

[[ 0  0  1  0  0  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  0  0 74  0  0  0]
 [ 0  0  0  0  0  0  1]]
[[0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]]
[[ 0  0  0  0  1  0  0]
 [ 0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  1]
 [ 0  1  0  0  0 78  0]]


In [22]:
def interpolate_column(col):
    xs = GF(np.array([1,2,3,4]))
    return galois.lagrange_poly(xs, col)

U_polys = np.apply_along_axis(interpolate_column, 0, L_galois)
V_polys = np.apply_along_axis(interpolate_column, 0, R_galois)
W_polys = np.apply_along_axis(interpolate_column, 0, O_galois)

print(U_polys)
print(V_polys)
print(W_polys)

[       0        0  6667130 20847764 20193579        0 33028635]
[       0        0  6667130 19502576 13779015        0        0]
[       0 33028635        0        0  6667130 26608143 19502576]


In [25]:
from functools import reduce

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y   # lambda functions
    add_ = lambda x,y: x+y

    return reduce(add_, map(mul_, polys, witness))

term_1 = inner_product_polynomials_with_witness(U_polys, witness)
term_2 = inner_product_polynomials_with_witness(V_polys, witness)
term_3 = inner_product_polynomials_with_witness(W_polys, witness)

print(term_1)
print(term_2)
print(term_3)


78x^3 + 76x^2 + 28x + 59
11x^3 + 77x^2 + 20x + 54
3x^3 + 40x^2 + 20x + 32


In [29]:
# t = (x-1)(x-2)(x-3)(x-4)

t= galois.Poly([1,78], field = GF) * galois.Poly([1,77], field = GF) * galois.Poly([1,76], field = GF) * galois.Poly([1,75], field = GF)
print(t)

h = (term_1 * term_2 - term_3) // t 

print(h)

assert term_1 * term_2 == term_3 + h * t, "division has a remainder"

x^4 + 69x^3 + 35x^2 + 29x + 24
68x^2 + 17x + 59
